In [396]:
import pandas as pd
import psycopg2
import numpy as np
import pymssql
import statsmodels.api as sm
import datetime
import warnings
warnings.filterwarnings('ignore')

#establishing the connection
conn = psycopg2.connect(
    database="iwdm", 
    user='stocke12', 
    password='Data5330', 
    host='data5330.cj2osauo8t14.us-west-1.rds.amazonaws.com', 
    port= '5432'
)
#Creating a cursor object using the cursor() method
cursor = conn.cursor()

In [291]:
# Creating all tables

cursor.execute('''
CREATE TABLE IF NOT EXISTS Customer (
customer_id INT PRIMARY KEY,
customer_fname VARCHAR(20) NOT NULL,
customer_lname VARCHAR(20) NOT NULL,
customer_age SMALLINT NOT NULL,
customer_type VARCHAR(10) NOT NULL,
customer_since SMALLINT NOT NULL,
customer_income INT NOT NULL,
household_size SMALLINT NOT NULL,
mode_color VARCHAR(10) NOT NULL
);
''')
conn.commit()

cursor.execute('''
CREATE TABLE IF NOT EXISTS NFL_TEAMS (
team_name VARCHAR(30) PRIMARY KEY,
team_name_short VARCHAR(15) NOT NULL,
team_id VARCHAR(4) NOT NULL,
team_id_pfr CHAR(3) NOT NULL,
team_conference CHAR(3) NOT NULL,
team_division VARCHAR(15),
team_conference_pre2002 CHAR(3) NOT NULL,
team_division_pre2002 VARCHAR(15)
);
''')
conn.commit()

cursor.execute('''
CREATE TABLE IF NOT EXISTS Stadium (
stadium_name VARCHAR(55) PRIMARY KEY,
stadium_location VARCHAR(30),
stadium_open SMALLINT,
stadium_close SMALLINT,
stadium_type VARCHAR(20),
stadium_address VARCHAR(100),
stadium_weather_station_code VARCHAR(30),
stadium_weather_type VARCHAR(20),
stadium_capacity INT,
stadium_surface VARCHAR(30),
station VARCHAR(20),
name VARCHAR(100),
latitude NUMERIC,
longitude NUMERIC,
elevation NUMERIC
);
''')
conn.commit()

cursor.execute('''
CREATE TABLE IF NOT EXISTS Spread_Scores (
game_id VARCHAR(30) PRIMARY KEY,
schedule_date DATE NOT NULL,
schedule_season SMALLINT NOT NULL,
schedule_week VARCHAR(30) NOT NULL,
schedule_playoff BOOLEAN NOT NULL,
team_home VARCHAR(30) NOT NULL,
home_score SMALLINT NOT NULL,
away_score SMALLINT NOT NULL,
team_away VARCHAR(30) NOT NULL,
team_favorite_id VARCHAR(5) NOT NULL,
spread_favorite NUMERIC NOT NULL,
over_under_line NUMERIC NOT NULL,
winner_ou VARCHAR(5),
winner_line CHAR(4),
stadium VARCHAR(50) NOT NULL,
stadium_neutral BOOLEAN NOT NULL,
stadium_temperature SMALLINT,
weather_wind_mph SMALLINT,
weather_humidity SMALLINT,
weather_detail VARCHAR(30)
);
''')
conn.commit()

cursor.execute('''
CREATE TABLE IF NOT EXISTS Betlog (
bet_id SMALLINT PRIMARY KEY,
customer_id SMALLINT NOT NULL,
game_id varchar(30) NOT NULL,
bet_amount NUMERIC NOT NULL,
bet_on VARCHAR(30) NOT NULL,
bet_result VARCHAR(5),
comm_value NUMERIC
);
''')
conn.commit()

In [214]:
# Adding data to the stadiums table

df = pd.read_csv("C:\\Users\\stock\\OneDrive\\Desktop\\DATA 5330\\Final Project\\nfl_stadiums.csv", encoding='latin1')
df['stadium_capacity'] = df['stadium_capacity'].str.replace(',', '')
df.fillna(0, inplace=True)
df['stadium_name'] = df['stadium_name'].str.replace("'", "")

for x in df.index:
    # Extract values from DataFrame
    stadium_open = df['stadium_open'].loc[x]
    stadium_close = df['stadium_close'].loc[x]
    stadium_capacity = df['stadium_capacity'].loc[x]
    # Convert string representations to appropriate types
    try:
        stadium_open = int(stadium_open)
    except:
        pass
    
    else:
        try:
            stadium_close = int(stadium_close)
        except:
            pass
    
    try:
        stadium_capacity = int(stadium_capacity)
    except:
        pass
    
    # Execute SQL query to insert data into the database
    cursor.execute('''
        INSERT INTO Stadium (stadium_name, stadium_location, stadium_open, stadium_close, stadium_type, stadium_address,
                             stadium_weather_station_code, stadium_weather_type, stadium_capacity, stadium_surface, station, name, latitude,
                             longitude, elevation)
        VALUES ('%s', '%s', %s, %s, '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s');
    ''' % (df['stadium_name'].loc[x], df['stadium_location'].loc[x], stadium_open, stadium_close, 
          df['stadium_type'].loc[x], df['stadium_address'].loc[x], df['stadium_weather_station_code'].loc[x], 
          df['stadium_weather_type'].loc[x], stadium_capacity, df['stadium_surface'].loc[x], 
          df['STATION'].loc[x], df['NAME'].loc[x], df['LATITUDE'].loc[x], df['LONGITUDE'].loc[x], df['ELEVATION'].loc[x]))
    
    # Commit changes to the database
    conn.commit()
    
    print('Record created for %s' % df['stadium_name'].loc[x])
    cursor.execute('''UPDATE stadium
                    SET stadium_location = NULL
                    WHERE stadium_location = '0'
                    ''')

    cursor.execute('''
                    UPDATE stadium
                    SET stadium_open = NULL
                    WHERE stadium_open = 0
    ''')

    cursor.execute('''
                    UPDATE stadium
                    SET stadium_close = NULL
                    WHERE stadium_close = 0
    ''')

    cursor.execute('''
                    UPDATE stadium
                    SET stadium_type = NULL
                    WHERE stadium_type = '0'
    ''')

    cursor.execute('''
                    UPDATE stadium
                    SET stadium_address = NULL
                    WHERE stadium_address = '0'
    ''')

    cursor.execute('''
                    UPDATE stadium
                    SET stadium_weather_station_code = NULL
                    WHERE stadium_weather_station_code = '0'
    ''')

    cursor.execute('''
                    UPDATE stadium
                    SET stadium_weather_type = NULL
                    WHERE stadium_weather_type = '0'
    ''')

    cursor.execute('''
                    UPDATE stadium
                    SET stadium_capacity = NULL
                    WHERE stadium_capacity = 0
    ''')

    cursor.execute('''
                    UPDATE stadium
                    SET stadium_surface = NULL
                    WHERE stadium_surface = '0'
    ''')

    cursor.execute('''
                    UPDATE stadium
                    SET station = NULL
                    WHERE station = '0'
    ''')

    cursor.execute('''
                    UPDATE stadium
                    SET name = NULL
                    WHERE name = '0'
    ''')

    cursor.execute('''
                    UPDATE stadium
                    SET latitude = NULL
                    WHERE latitude = 0
    ''')

    cursor.execute('''
                    UPDATE stadium
                    SET longitude = NULL
                    WHERE longitude = 0
    ''')

    cursor.execute('''
                    UPDATE stadium
                    SET elevation = NULL
                    WHERE elevation = 0
    ''')
    conn.commit()

 


Record created for Acrisure Stadium
Record created for Alamo Dome
Record created for Allegiant Stadium
Record created for Allianz Arena
Record created for Alltel Stadium
Record created for Alumni Stadium
Record created for Anaheim Stadium
Record created for Arrowhead Stadium
Record created for AT&T Stadium
Record created for Atlanta-Fulton County Stadium
Record created for Balboa Stadium
Record created for Bank of America Stadium
Record created for Bills Stadium
Record created for Busch Memorial Stadium
Record created for Caesars Superdome
Record created for Candlestick Park
Record created for CenturyLink Field
Record created for Cinergy Field
Record created for Cleveland Municipal Stadium
Record created for Cotton Bowl
Record created for Cowboys Stadium
Record created for Dolphin Stadium
Record created for Edward Jones Dome
Record created for Empower Field at Mile High
Record created for Estadio Azteca
Record created for EverBank Field
Record created for FedEx Field
Record created for

In [164]:
# Addings data to the teams table

df = pd.read_csv("C:\\Users\\stock\\OneDrive\\Desktop\\DATA 5330\\Final Project\\nfl_teams.csv", index_col='team_name')
for x in df.index:
    cursor.execute('''
    INSERT INTO NFL_Teams (team_name, team_name_short, team_id, team_id_pfr, team_Conference, team_division,
    team_conference_pre2002, team_division_pre2002)
    VALUES ('%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s');
    ''' % (x, df['team_name_short'].loc[x], df['team_id'].loc[x], df['team_id_pfr'].loc[x], 
          df['team_conference'].loc[x], df['team_division'].loc[x], df['team_conference_pre2002'].loc[x],
           df['team_division_pre2002'].loc[x]))
    conn.commit()
    print('Record created for %s' % x)
    cursor.execute('''
    UPDATE NFL_Teams
    SET team_division = NULL,
        team_division_pre2002 = NULL
    WHERE team_division = 'nan'
          or team_division_pre2002 = 'nan';
    ''')
    conn.commit

Record created for Arizona Cardinals
Record created for Atlanta Falcons
Record created for Baltimore Colts
Record created for Baltimore Ravens
Record created for Boston Patriots
Record created for Buffalo Bills
Record created for Carolina Panthers
Record created for Chicago Bears
Record created for Cincinnati Bengals
Record created for Cleveland Browns
Record created for Dallas Cowboys
Record created for Denver Broncos
Record created for Detroit Lions
Record created for Green Bay Packers
Record created for Houston Oilers
Record created for Houston Texans
Record created for Indianapolis Colts
Record created for Jacksonville Jaguars
Record created for Kansas City Chiefs
Record created for Las Vegas Raiders
Record created for Los Angeles Chargers
Record created for Los Angeles Raiders
Record created for Los Angeles Rams
Record created for Miami Dolphins
Record created for Minnesota Vikings
Record created for New England Patriots
Record created for New Orleans Saints
Record created for New

In [311]:
# Adding data to the spread_scores table

df = pd.read_csv("C:\\Users\\stock\\OneDrive\\Desktop\\DATA 5330\\Final Project\\spread_scores-1.csv")
df.fillna(10000, inplace=True)
df['schedule_date'] = pd.to_datetime(df['schedule_date'])
df = df[df['schedule_date'] >= '2015-09-01']
df['stadium'] = df['stadium'].str.replace("'", "")
conditions = [((df['score_home'] + df['score_away']) > df['over_under_line']),
             ((df['score_home'] + df['score_away']) < df['over_under_line']),
             ((df['score_home'] + df['score_away']) == df['over_under_line'])]
values = ['Over', 'Under', 'Push']
df.insert(loc=11, column='winner_ou', value=pd.np.select(conditions, values))
# conditions = [(df['score_home'] > df['score_away']),
#              (df['score_home'] < df['score_away']),
#              (df['score_home'] == df['score_away'])]
# values = ['Home', 'Away', 'Push']
# df.insert(loc=12, column='winner_line', value=pd.np.select(conditions, values))
df.insert(loc=0, column='game_id', value=None)
df['schedule_week'].replace({'1':'01', '2':'02', '3':'03', '4':'04', '5':'05', '6':'06', '7':'07', '8':'08', '9':'09', 
                             'Wildcard':'19', 'Division':'20', 'Conference':21, 'Superbowl':22}, inplace=True)

query = '''
SELECT *
FROM nfl_teams;
'''
df2 = pd.read_sql(query, conn)

for x in df.index:
    for y in df2.index:
        if df['team_home'].loc[x] == df2['team_name'].loc[y]:
            home = df2['team_id'].loc[y]
        elif df['team_away'].loc[x] == df2['team_name'].loc[y]:
            away = df2['team_id'].loc[y]
            
        if df['team_favorite_id'].loc[x] == home and (df['score_home'].loc[x] - df['score_away'].loc[x]) > (df['spread_favorite'].loc[x] * -1):
            winner = 'home'
        elif df['team_favorite_id'].loc[x] == home and (df['score_home'].loc[x] - df['score_away'].loc[x]) < (df['spread_favorite'].loc[x] * -1):
            winner = 'away'
        elif df['team_favorite_id'].loc[x] == away and (df['score_away'].loc[x] - df['score_home'].loc[x]) > (df['spread_favorite'].loc[x] * -1):
            winner = 'away'
        elif df['team_favorite_id'].loc[x] == away and (df['score_away'].loc[x] - df['score_home'].loc[x]) < (df['spread_favorite'].loc[x] * -1):
            winner = 'home'
        else:
            winner = 'push'
        
    stadium_temp = int(df['weather_temperature'].loc[x])
    wind = int(df['weather_wind_mph'].loc[x])
    humidity = int(df['weather_humidity'].loc[x])
    game_id = '%s%s-%s-%s' % (df['schedule_season'].loc[x], df['schedule_week'].loc[x], home, away)
    df['game_id'].loc[x] = game_id
    cursor.execute('''
    INSERT INTO spread_scores (game_id, schedule_date, schedule_season, schedule_week, schedule_playoff, team_home, home_score,
    away_score, team_away, team_favorite_id, spread_favorite, over_under_line, winner_ou, winner_line,
    stadium, stadium_neutral, stadium_temperature, weather_wind_mph, weather_humidity, weather_detail)
    VALUES ('%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s')
    ''' % (game_id, df['schedule_date'].loc[x], df['schedule_season'].loc[x], df['schedule_week'].loc[x], df['schedule_playoff'].loc[x],
    df['team_home'].loc[x], df['score_home'].loc[x], df['score_away'].loc[x], df['team_away'].loc[x],
    df['team_favorite_id'].loc[x], df['spread_favorite'].loc[x], df['over_under_line'].loc[x],
    df['winner_ou'].loc[x], winner, df['stadium'].loc[x], df['stadium_neutral'].loc[x], 
    stadium_temp, wind, humidity, 
    df['weather_detail'].loc[x]))
    conn.commit()
    print('Record Created for %s' % game_id)
    
    cursor.execute('''
                    UPDATE spread_scores
                    SET stadium_temperature = NULL
                    WHERE stadium_temperature = '10000'
    ''')
    cursor.execute('''
                    UPDATE spread_scores
                    SET weather_wind_mph = NULL
                    WHERE weather_wind_mph = '10000'
    ''')
    cursor.execute('''
                    UPDATE spread_scores
                    SET weather_humidity = NULL
                    WHERE weather_humidity = '10000'
    ''')
    cursor.execute('''
                    UPDATE spread_scores
                    SET weather_detail = NULL
                    WHERE weather_detail = '10000'
    ''')
    conn.commit()
    

Record Created for 201501-NE-PIT
Record Created for 201501-ARI-NO
Record Created for 201501-BUF-IND
Record Created for 201501-CHI-GB
Record Created for 201501-DAL-NYG
Record Created for 201501-DEN-BAL
Record Created for 201501-HOU-KC
Record Created for 201501-JAX-CAR
Record Created for 201501-NYJ-CLE
Record Created for 201501-LVR-CIN
Record Created for 201501-LAC-DET
Record Created for 201501-LAR-SEA
Record Created for 201501-TB-TEN
Record Created for 201501-WAS-MIA
Record Created for 201501-ATL-PHI
Record Created for 201501-SF-MIN
Record Created for 201502-KC-DEN
Record Created for 201502-BUF-NE
Record Created for 201502-CAR-HOU
Record Created for 201502-CHI-ARI
Record Created for 201502-CIN-LAC
Record Created for 201502-CLE-TEN
Record Created for 201502-GB-SEA
Record Created for 201502-JAX-MIA
Record Created for 201502-MIN-DET
Record Created for 201502-NO-TB
Record Created for 201502-NYG-ATL
Record Created for 201502-LVR-BAL
Record Created for 201502-PHI-DAL
Record Created for 201502

In [345]:
msconn = pymssql.connect(
     host='stairwaytoheaven.usu.edu', # Server name goes in quotes.
     user='5330user', # Username goes in quotes.
     password='pipelinesnow', # Password goes in quotes
     database='ironwill') # Database to use goes in quotes

mscursor = conn.cursor()

In [178]:
# Adding data to the customer table
query = '''
SELECT *
FROM customer_table
'''
df = pd.read_sql(query, msconn, index_col='customer_id')
df[['fname', 'lname']] = df['customer_name'].str.split(' ', 1, expand=True)
df['fname'] = df['fname'].str.replace("'", "")
df['lname'] = df['lname'].str.replace("'", "")

for x in df.index:
    cursor.execute('''
    INSERT INTO customer (customer_id, customer_fname, customer_lname, customer_age, customer_type, customer_since,
                          customer_income, household_size, mode_color)
    VALUES ('%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s')
    ''' % (x, df['fname'].loc[x], df['lname'].loc[x], df['customer_age'].loc[x], df['customer_type'].loc[x], df['customer_since'].loc[x], 
          df['customer_income'].loc[x], df['household_size'].loc[x], df['mode_color'].loc[x]))
    conn.commit()
    print('Record created for %s' % x)

Record created for 1
Record created for 2
Record created for 3
Record created for 4
Record created for 5
Record created for 6
Record created for 7
Record created for 8
Record created for 9
Record created for 10
Record created for 11
Record created for 12
Record created for 13
Record created for 14
Record created for 15
Record created for 16
Record created for 17
Record created for 18
Record created for 19
Record created for 20
Record created for 21
Record created for 22
Record created for 23
Record created for 24
Record created for 25
Record created for 26
Record created for 27
Record created for 28
Record created for 29
Record created for 30
Record created for 31
Record created for 32
Record created for 33
Record created for 34
Record created for 35
Record created for 36
Record created for 37
Record created for 38
Record created for 39
Record created for 40
Record created for 41
Record created for 42
Record created for 43
Record created for 44
Record created for 45
Record created for 

In [346]:
# Adding data to the betlog table
query = '''
SELECT *
FROM betlog;
'''
x = 0
df = pd.read_sql(query, msconn, index_col='bet_id')
df = df[df.index > 57810]

for x in df.index:
    cursor.execute('''
    INSERT INTO betlog (bet_id, customer_id, game_id, bet_amount, bet_on)
    VALUES ('%s', '%s', '%s', '%s', '%s')
    ''' % (x, df['customer_id'].loc[x], df['game_id'].loc[x], df['bet_amount'].loc[x], df['bet_on'].loc[x]))
    conn.commit()

KeyboardInterrupt: 

In [360]:
query = '''
SELECT bet_id, bet_on, team_home, team_away, winner_line, winner_ou, bet_amount
FROM betlog b
JOIN spread_scores s ON TRIM(b.game_id) = s.game_id
WHERE bet_result IS NULL
ORDER BY 1;
'''

df = pd.read_sql(query, conn, index_col = 'bet_id')
df['bet_result'] = ''
df['comm_value'] = ''
for x in df.index:
    if df['bet_on'].loc[x] == df['team_home'].loc[x]:
        bet = 'Home'
        if df['winner_line'].loc[x] == bet:
            df['bet_result'].loc[x] = 'win'
        else:
            df['bet_result'].loc[x] = 'loss'
    elif df['bet_on'].loc[x] == df['team_away'].loc[x]:
        bet = 'away'
        if df['winner_line'].loc[x] == bet:
            df['bet_result'].loc[x] = 'win'
        else:
            df['bet_result'].loc[x] = 'loss'
    elif df['bet_on'].loc[x] == 'over':
        if df['winner_ou'].loc[x] == 'Over':
            df['bet_result'].loc[x] = 'win'
        elif df['winner_ou'].loc[x] == 'Under':
            df['bet_result'].loc[x] = 'loss'
        elif df['winner_ou'].loc[x] == 'push':
            df['bet_result'].loc[x] = 'push'
    elif df['bet_on'].loc[x] == 'under':
        if df['winner_ou'].loc[x] == 'Under':
            df['bet_result'].loc[x] = 'win'
        elif df['winner_ou'].loc[x] == 'Over':
            df['bet_result'].loc[x] = 'loss'
        elif df['winner_ou'].loc[x] == 'push':
            df['bet_result'].loc[x] = 'push'
    elif df['winner_line'].loc[x] == 'push':
        df['bet_result'] = 'push'
        
    if df['bet_amount'].loc[x] <= 1000:
        df['comm_value'].loc[x] = df['bet_amount'].loc[x] * 0.1
    elif df['bet_amount'].loc[x] <= 4000:
        df['comm_value'].loc[x] = (df['bet_amount'].loc[x] - 1000) * 0.08 + 100
    else:
        df['comm_value'].loc[x] = (df['bet_amount'].loc[x] - 5000) * 0.06 + 420
        
    cursor.execute('''
        UPDATE betlog
        SET bet_result = '%s'
        WHERE bet_id = '%s' AND bet_result IS NULL;
    ''' % (df['bet_result'].loc[x], x))
    cursor.execute('''
        UPDATE betlog
        SET comm_value = '%s'
        WHERE bet_id = '%s' AND comm_value IS NULL;
    ''' % (df['comm_value'].loc[x], x))
    conn.commit()
print('All records created')

All records created


In [361]:
# QUERIES


In [331]:
# 1
query = '''
SELECT *
FROM betlog
ORDER BY 1
LIMIT 5;
'''
df = pd.read_sql(query, conn)
print(df)
print('-----------------------')

query = '''
SELECT *
FROM customer
ORDER BY 1
LIMIT 5;
'''
df = pd.read_sql(query, conn)
print(df)
print('-----------------------')

query = '''
SELECT *
FROM nfl_teams
ORDER BY 1
LIMIT 5;
'''
df = pd.read_sql(query, conn)
print(df)
print('-----------------------')

query = '''
SELECT *
FROM spread_scores
ORDER BY 1
LIMIT 5;
'''
df = pd.read_sql(query, conn)
print(df)
print('-----------------------')

query = '''
SELECT *
FROM stadium
ORDER BY 1
LIMIT 5;
'''
df = pd.read_sql(query, conn)
print(df)

   bet_id  customer_id         game_id  bet_amount            bet_on  \
0       1            5  202201-LAR-BUF       100.0  Los Angeles Rams   
1       2            6  202201-LAR-BUF       350.0  Los Angeles Rams   
2       3           22  202201-LAR-BUF      2000.0  Los Angeles Rams   
3       4           24  202201-LAR-BUF      6000.0  Los Angeles Rams   
4       5           26  202201-LAR-BUF       100.0     Buffalo Bills   

  bet_result  comm_value  
0       loss        10.0  
1       loss        35.0  
2       loss       180.0  
3       loss       480.0  
4        win        10.0  
-----------------------
   customer_id customer_fname customer_lname  customer_age customer_type  \
0            1        Leilani          Garza            31        online   
1            2       Samantha         Curtis            45         phone   
2            3        Matthew        Morales            31         local   
3            4         Holden        Randall            34        online   
4

In [367]:
# 2
query = '''
WITH betcte AS (
SELECT customer_id, SUM(comm_value) TotalComm
FROM betlog
GROUP BY customer_id
HAVING SUM(comm_value) > 20000
)

SELECT COUNT(DISTINCT bc.customer_id) BigBettors, COUNT(DISTINCT b.customer_id) TotalBettors,
        CAST(COUNT(DISTINCT bc.customer_id) AS DECIMAL) * 100 / COUNT(DISTINCT b.customer_id) PercBigBettors
FROM betcte bc
RIGHT OUTER JOIN betlog b ON bc.customer_id = b.customer_id;
'''
df = pd.read_sql(query, conn)
print(df)

query = '''
SELECT customer_fname, customer_lname, SUM(comm_value) CommPaid
FROM customer c
JOIN betlog b ON c.customer_id = b.customer_id
GROUP BY customer_fname, customer_lname
ORDER BY 3 DESC
LIMIT 20;
'''

df = pd.read_sql(query, conn)
print(df)

   bigbettors  totalbettors  percbigbettors
0         241          2000           12.05
   customer_fname customer_lname  commpaid
0            Neal           lark  189052.0
1            Zoie    Blankenship  177886.5
2          Chanel         Martin  135945.0
3          Gunner         Morris  134229.0
4           Amari       Chandler  134162.5
5         Jackson          White  132301.0
6           Jacob       Gonzalez  131272.0
7          Xander          Jones  124731.0
8           Miles          Young  121371.5
9          Steven          Brown  119022.5
10         Xavier           Lowe  118042.5
11         Harvey          Ayers  115461.5
12         Harris       Campbell  114158.0
13        Eleanor         Torres  107990.0
14          Ariah         OBrien  105516.0
15           Elsa          Henry   99414.0
16         Hallie         Patton   95706.0
17        Delilah        Nichols   94341.0
18         Amirah         Taylor   90730.0
19          Micah      Davenport   90387.5


In [368]:
# 3
query = '''
WITH wincte AS (
    SELECT customer_id, COUNT(*) AS won_count, SUM(bet_amount) AS TotalAmount
    FROM betlog
    WHERE bet_result = 'win'
    GROUP BY customer_id
    )
    
SELECT c.customer_fname, c.customer_lname, COUNT(b.*) AS bets_placed, w.won_count AS bets_won,
       w.won_count * 100.0 / COUNT(b.*) AS winning_percentage, w.TotalAmount
FROM betlog b
JOIN customer c ON c.customer_id = b.customer_id
JOIN wincte w ON c.customer_id = w.customer_id
GROUP BY c.customer_fname, c.customer_lname, w.won_count, w.TotalAmount
HAVING COUNT(b.*) >= 6
ORDER BY winning_percentage DESC, totalamount DESC
LIMIT 10;
'''

df = pd.read_sql(query, conn)
print(df)

  customer_fname customer_lname  bets_placed  bets_won  winning_percentage  \
0        Spencer     Fitzgerald           13        12           92.307692   
1          Anika       Ferguson            8         7           87.500000   
2           Noel          Ramos           12         9           75.000000   
3        Azariah         Sexton           12         9           75.000000   
4          Paris        Schmidt           11         8           72.727273   
5        Everlee       Anderson            7         5           71.428571   
6           Cory           Lamb           37        26           70.270270   
7          Aiden           Mann           10         7           70.000000   
8           Lila         Rivera           15        10           66.666667   
9        Lorenzo          Moran            6         4           66.666667   

   totalamount  
0       1375.0  
1        950.0  
2      21850.0  
3       1050.0  
4        900.0  
5        675.0  
6     180200.0  
7    

In [370]:
# 4

query = '''
WITH WinCTE AS (
  SELECT customer_id, COUNT(*) BetsWon, SUM(bet_amount) TotalPaidOut
  FROM betlog
  WHERE bet_result = 'win'
  GROUP BY customer_id),
  
 LossCTE AS (
   SELECT customer_id, SUM(bet_amount) + SUM(comm_value) TotalEarned
   FROM betlog
   WHERE bet_result = 'loss'
   GROUP BY customer_id
)

SELECT customer_fname, customer_lname, COUNT(b.*) TotalBets, BetsWon, (TotalPaidOut - TotalEarned) NetLoss
FROM customer c
JOIN betlog b ON c.customer_id = b.customer_id
JOIN WinCTE w ON c.customer_id = w.customer_id
JOIN LossCTE l ON c.customer_id = l.customer_id
GROUP BY customer_fname, customer_lname, BetsWon, TotalPaidOut, TotalEarned
ORDER BY NetLoss DESC
LIMIT 20;
'''

df = pd.read_sql(query, conn)
print(df)

   customer_fname customer_lname  totalbets  betswon  netloss
0            Cory           Lamb         37       26  94891.0
1         Octavia           Moss        104       58  76118.0
2            Ella          Reyes         21       11  30777.0
3             Han          Hayes         49       25  30200.0
4           Emory           Hunt        131       74  28593.0
5         Valerie         Muller         12        4  23092.0
6          Nathan      McCormick         63       35  20807.0
7         Kaylani          Cross         60       34  17136.0
8            Noel          Ramos         12        9  16390.0
9            Zane           Bell         12        7  11214.0
10         Harper     Cunningham         10        6  10790.0
11        Lincoln           Bond         10        3   9597.5
12       Benjamin          Lopez         11        6   9218.0
13           Ella         Martin        129       73   8537.5
14           Lila         Rivera         15       10   8020.0
15      

In [376]:
# 5
query = '''
WITH AmtLost AS (
SELECT game_id, SUM(bet_amount) AmountLost
FROM betlog
WHERE bet_result = 'win'
GROUP BY game_id
ORDER BY 1),
AmtEarned AS (
SELECT game_id, SUM(bet_amount) AmountEarned
FROM betlog
WHERE bet_result = 'loss'
GROUP BY game_id
ORDER BY 1),
Outcome AS (
  SELECT schedule_week, SUM(CASE WHEN AmountEarned > AmountLost THEN 1 ELSE 0 END) AS GamesWon,
  			 SUM(CASE WHEN AmountEarned < AmountLost THEN 1 ELSE 0 END) AS GamesLost
  FROM spread_scores s
  JOIN AmtLost l ON s.game_id = TRIM(l.game_id)
  JOIN AmtEarned e ON s.game_id = TRIM(e.game_id)
  GROUP BY schedule_week)
  
SELECT s.schedule_week, COUNT(DISTINCT s.game_id) as TotalGames, GamesWon, GamesLost,
			 GamesWon * 100.0 / COUNT(DISTINCT s.game_id) as PercWon,
       GamesLost * 100.0 / COUNT(DISTINCT s.game_id) as PercLost
FROM spread_scores s
JOIN Outcome o ON o.schedule_week = s.schedule_week
WHERE schedule_season = 2022
GROUP BY s.schedule_week, GamesWon, GamesLost
ORDER BY 1;
'''

df = pd.read_sql(query, conn)
print(df)

   schedule_week  totalgames  gameswon  gameslost     percwon   perclost
0             01          16        15          1   93.750000   6.250000
1             02          16        16          0  100.000000   0.000000
2             03          16        14          2   87.500000  12.500000
3             04          16        16          0  100.000000   0.000000
4             05          16        14          2   87.500000  12.500000
5             06          14        13          1   92.857143   7.142857
6             07          14        13          1   92.857143   7.142857
7             08          15        12          2   80.000000  13.333333
8             09          13        13          0  100.000000   0.000000
9             10          14        13          1   92.857143   7.142857
10            11          14        14          0  100.000000   0.000000
11            12          16        15          1   93.750000   6.250000
12            13          15        13          2  

In [403]:
# 6
query = '''
SELECT nfl_teams.team_name, home_wins + away_wins AS wins, loss_counts.losses, spread_win_counts.spread_wins,
bets_for_counts.bets_for, bets_against_counts.bets_against
FROM nfl_teams
LEFT JOIN (
SELECT team_home AS team_name, COUNT(*) AS home_wins
FROM spread_scores
WHERE schedule_season = 2022 AND winner_ou = 'Over'
GROUP BY team_home
) AS home_win_counts ON nfl_teams.team_name = home_win_counts.team_name
LEFT JOIN (
SELECT team_away AS team_name, COUNT(*) AS away_wins
FROM spread_scores
WHERE schedule_season = 2022 AND winner_ou = 'Under'
GROUP BY team_away
) AS away_win_counts ON nfl_teams.team_name = away_win_counts.team_name
LEFT JOIN (SELECT team_home AS team_name, COUNT(*) AS losses
FROM spread_scores
WHERE schedule_season = 2022 AND winner_ou = 'Under'
GROUP BY team_home
) AS loss_counts ON nfl_teams.team_name = loss_counts.team_name
LEFT JOIN (SELECT team_home AS team_name, COUNT(*) AS spread_wins
FROM spread_scores
WHERE schedule_season = 2022 AND ((winner_ou = 'Over' AND winner_line = 'home') OR (winner_ou = 'Under' AND winner_line = 'away'))
GROUP BY team_home
) AS spread_win_counts ON nfl_teams.team_name = spread_win_counts.team_name
LEFT JOIN (SELECT bet_on AS team_name, COUNT(*) AS bets_for
FROM betlog
GROUP BY bet_on
) AS bets_for_counts ON nfl_teams.team_name = bets_for_counts.team_name
LEFT JOIN (SELECT bet_on AS team_name, COUNT(*) AS bets_against
FROM betlog
WHERE bet_result = 'loss'
GROUP BY bet_on
) AS bets_against_counts ON nfl_teams.team_name = bets_against_counts.team_name
ORDER BY nfl_teams.team_name;
'''
df = pd.read_sql(query, conn)
print(df)

                   team_name  wins  losses  spread_wins  bets_for  \
0          Arizona Cardinals   8.0     4.0          4.0    2886.0   
1            Atlanta Falcons   9.0     5.0          6.0    2889.0   
2            Baltimore Colts   NaN     NaN          NaN       NaN   
3           Baltimore Ravens   6.0     7.0          5.0    2931.0   
4            Boston Patriots   NaN     NaN          NaN       NaN   
5              Buffalo Bills  13.0     4.0          6.0    3179.0   
6          Carolina Panthers   8.0     5.0          2.0    2868.0   
7              Chicago Bears   8.0     4.0          3.0    2912.0   
8         Cincinnati Bengals  11.0     4.0          3.0    3207.0   
9           Cleveland Browns   9.0     3.0          2.0    2885.0   
10            Dallas Cowboys  11.0     4.0          4.0    3143.0   
11            Denver Broncos   9.0     5.0          6.0    2914.0   
12             Detroit Lions  10.0     3.0          4.0    2789.0   
13         Green Bay Packers  10.0

In [382]:
# Test Analysis

In [392]:
#7a
query = '''
SELECT customer_age, customer_type, customer_income, household_size
FROM betlog b
JOIN customer c ON b.customer_id = c.customer_id;
'''
df = pd.read_sql(query, conn)
df['isLocal'] = np.where(df['customer_type'] == 'local', 1, 0)
df['isOnline'] = np.where(df['customer_type'] == 'online', 1, 0)
df['isPhone'] = np.where(df['customer_type'] == 'phone', 1, 0)
correlation_matrix = df.corr()
print(correlation_matrix)
print('The only variables with a high correlation are isLocal and isOnline. This is expected considering they are opposites')


                 customer_age  customer_income  household_size   isLocal  \
customer_age         1.000000        -0.029157        0.009164 -0.030218   
customer_income     -0.029157         1.000000       -0.022651  0.009998   
household_size       0.009164        -0.022651        1.000000 -0.022713   
isLocal             -0.030218         0.009998       -0.022713  1.000000   
isOnline             0.034864         0.004102        0.001154 -0.829167   
isPhone             -0.005262        -0.024288        0.037746 -0.366194   

                 isOnline   isPhone  
customer_age     0.034864 -0.005262  
customer_income  0.004102 -0.024288  
household_size   0.001154  0.037746  
isLocal         -0.829167 -0.366194  
isOnline         1.000000 -0.216536  
isPhone         -0.216536  1.000000  
The only variables with a high correlation are isLocal and isOnline. This is expected considering they are opposites


In [397]:
#7b
query = '''
WITH WinCTE AS (
SELECT customer_id, SUM(bet_amount) as AmountWon
FROM betlog
WHERE bet_result = 'win'
GROUP BY customer_id
)
SELECT b.customer_id, customer_age, customer_type, customer_income, household_size,
			 (SUM(bet_amount) + SUM(comm_value)) - AmountWon as Value
FROM customer c
JOIN betlog b ON c.customer_id = b.customer_id
JOIN WinCTE w ON b.customer_id = w.customer_id
GROUP BY b.customer_id, customer_age, customer_type, customer_income, household_size, AmountWon;
'''
df = pd.read_sql(query, conn)
df['isLocal'] = np.where(df['customer_type'] == 'local', 1, 0)
df['isOnline'] = np.where(df['customer_type'] == 'online', 1, 0)
x = df[['customer_age', 'customer_income', 'household_size', 'isLocal', 'isOnline']]
y = df['value']
x = sm.add_constant(x)

model = sm.OLS(y, x).fit()
predictions = model.predict(x)

print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.103
Model:                            OLS   Adj. R-squared:                  0.101
Method:                 Least Squares   F-statistic:                     44.51
Date:                Wed, 24 Apr 2024   Prob (F-statistic):           1.40e-43
Time:                        09:46:52   Log-Likelihood:                -26330.
No. Observations:                1938   AIC:                         5.267e+04
Df Residuals:                    1932   BIC:                         5.270e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const            -7.37e+04   2.58e+04     

In [402]:
query = '''
WITH WinCTE AS (
SELECT customer_id, SUM(bet_amount) as AmountWon
FROM betlog
WHERE bet_result = 'win'
GROUP BY customer_id
)
SELECT b.customer_id, customer_age, customer_type, customer_income, household_size, mode_color,
			 (SUM(bet_amount) + SUM(comm_value)) - AmountWon as Value
FROM customer c
JOIN betlog b ON c.customer_id = b.customer_id
JOIN WinCTE w ON b.customer_id = w.customer_id
GROUP BY b.customer_id, customer_age, customer_type, customer_income, household_size, mode_color, AmountWon;
'''
df = pd.read_sql(query, conn)
df['isLocal'] = np.where(df['customer_type'] == 'local', 1, 0)
df['isOnline'] = np.where(df['customer_type'] == 'online', 1, 0)
df['isRed'] = np.where(df['mode_color'] == 'red', 1, 0)
df['isPurple'] = np.where(df['mode_color'] == 'purple', 1, 0)
df['isOrange'] = np.where(df['mode_color'] == 'orange', 1, 0)
df['isGreen'] = np.where(df['mode_color'] == 'green', 1, 0)
df['isBlue'] = np.where(df['mode_color'] == 'blue', 1, 0)
df['isBlack'] = np.where(df['mode_color'] == 'black', 1, 0)
df['isYellow'] = np.where(df['mode_color'] == 'yellow', 1, 0)

x = df[['customer_income', 'household_size', 'isOnline', 'isPurple', 'isOrange', 'isGreen', 'isBlue', 'isBlack', 'isYellow']]
y = df['value']
x = sm.add_constant(x)

model = sm.OLS(y, x).fit()
predictions = model.predict(x)

print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.130
Model:                            OLS   Adj. R-squared:                  0.126
Method:                 Least Squares   F-statistic:                     32.07
Date:                Wed, 24 Apr 2024   Prob (F-statistic):           7.61e-53
Time:                        09:57:00   Log-Likelihood:                -26300.
No. Observations:                1938   AIC:                         5.262e+04
Df Residuals:                    1928   BIC:                         5.268e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const           -2.523e+04   1.43e+04     